In [177]:
import os
import sys
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from nltk.stem import PorterStemmer
# import pyodbc
import collections
from sklearn.tree import DecisionTreeRegressor
DT = DecisionTreeRegressor()

In [178]:
data = pd.read_excel('Y Hospital.xls',header=1)
data1 = data.copy()
len(data1)
data1.head()

,Service Code,Service Name,Service Type,Price
0,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,47.25
1,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,7575.00
2,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,12755.00
3,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,8558.00
4,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,13550.00


In [179]:
print(len(data1[data1['Service Name'].str.contains('DAY PKG')]))
print(len(data1[data1['Service Name'].str.contains('DAYS')]))
#print(len(ref1[ref1['LongDescription'].str.contains('CONSULTATION')]))
print(len(data1[data1['Service Name'].isna()]))
#print(len(ref1[ref1['LongDescription'].isna()]))
#ref[ref['LongDescription'].str.contains('Scan')]
#ref[ref['LongDescription'].str.contains('CT')]
#ref[ref['LongDescription'].str.contains('C.T')]

1006
504
0


In [180]:
data1['Service Name'] = data1['Service Name'].str.replace('DAY PKG', '')
data1['Service Name'] = data1['Service Name'].str.replace('DAYS', '')
data1.head()

,Service Code,Service Name,Service Type,Price
0,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,47.25
1,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,7575.00
2,07300200,CHOLECYSTECTOMY - 5,Package Deal,12755.00
3,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1,Package Deal,8558.00
4,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,13550.00


In [181]:
tf1 = TfidfVectorizer()
data1['Service Name'] = data1['Service Name'].apply(lambda x: ''.join(i for i in x if not i.isdigit()))
tf1.fit_transform(data1['Service Name'])
print(len(tf1.get_feature_names()))
data_features = pd.DataFrame(tf1.get_feature_names(), columns = ['Features'])
data_features.head()

9653


,Features
0,ab
1,abc
2,abca
3,abces
4,abcess


In [182]:
service_name = pd.DataFrame(data1['Service Name'])

In [183]:
service_name.head()

,Service Name
0,LANOXIN-.MG INJECTION-'S
1,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...
2,CHOLECYSTECTOMY -
3,UNILATERAL REPAIR OF INGUINAL HERNIA -
4,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...


In [184]:
ACHI = pd.read_excel('ACHI_WITH_BLOCK_MBS.xlsx')

In [185]:
ACHI.head()

,Serv,ACHI,MBS,Block
0,Achillotenotomy 49718-01 [1542],Achillotenotomy,49718-01,[1542]
1,Acromioplasty (open) 48903-00 [1400],Acromioplasty (open),48903-00,[1400]
2,Acupuncture 93173-00 [1908],Acupuncture,93173-00,[1908]
3,"Adaptometry, dark 11211-00 [1831]","Adaptometry, dark",11211-00,[1831]
4,Adenoidectomy 41801-00 [412],Adenoidectomy,41801-00,[412]


In [186]:
def create_document(DF1,col_index): #DF1 is the DF to be preprocessed #col_index is the number of the describtion column (int)
    # Making A copy to prevent change to the original DF as it is needd subsequently
    DF = DF1.copy()
    # Insuring string type for the wanted col
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].astype(str)
    # Removing all newline and replacing it with space
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].str.replace('\n',' ')
    #Replacing all non alphanumeric in both english and arabic by space
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: re.sub('[^0-9a-zA-Z\u0627-\u064a]+', ' ', x))
    #Separating camel case with space 
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: re.sub(r"([a-z])([A-Z])", r"\1 \2", x))
    #lowercasing all describtions
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x.lower() for x in x.split()))
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x:''.join(i for i in x if not i.isdigit()))
    #In case a med dict present (referring the medical abbreviation to its origin) this line return the abbreviation to origin
    ## DF_document.document = DF_document.document.apply(lambda x: " ".join(med_dict(word) for word in x.split()) )
    #Counting all Words that occur either in very low freq or very high and removing them (Threshhold must be configured)
    ## freq = pd.Series(' '.join(DF[DF.columns[col_index]]).split()).value_counts()[:50]
    ## DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x for x in x.split() if  x not in freq.index))
    ## freq = pd.Series(' '.join(DF[DF.columns[col_index]]).split()).value_counts()[-1034:]
    ## DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join(x for x in x.split() if x not in freq.index))
    #Defining a stemmer and stemming all words in the description
    st = PorterStemmer()
    #DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    DF[DF.columns[col_index]] = DF[DF.columns[col_index]].apply(lambda x: "" if x.isdigit() == True else x)
    return DF

In [187]:
#Defining a function that take a single row string as first argument and the DF to be looked upon 
#this function return the index of service which has an exact match in the lookup or Null if there is not one
def dup_row_refrence(sd_desc,unique_list):
    for index,value in unique_list.items():
        if sd_desc == value:
            return index
    return 'NaN'

In [188]:
#A function that uses dup_row_refrence by taking 1 DF and refrencing it by its self
def dup_dup(DF_dup, col_name):
    #Dropping duplicate from DF to create a refrence (drop duplicate except First)
    DF_unique =  DF_dup[col_name].drop_duplicates()
    #getting the subset of the services of processed DF that occur in refrence (all except first also)
    DF_dup = DF_dup[DF_dup.duplicated(subset = col_name)]
    #applying dup_row_refrence for each row
    dup_ref = DF_dup[col_name].apply(lambda x: dup_row_refrence(x,DF_unique))
    return dup_ref


In [189]:
# A function that uses dup_row_refrence by taking 2 DF and refrencing one by the other
def dup_ref(DF_ref,DF_dup):
    #Dropping duplicate from refrence
    DF_unique =  DF_ref["LongDescription"].drop_duplicates()
    #getting the subset of the services of processed DF that occur in refrence
    DF_dup = DF_dup[DF_dup["Service Name"].isin(DF_unique)]
    #applying dup_row_refrence for each row
    dup_ref = DF_dup['Service Name'].apply(lambda x: dup_row_refrence(x,DF_unique))
    return dup_ref

In [190]:
ACHI_processed = create_document(ACHI, 1)

In [191]:
ACHI_processed[ACHI_processed.columns[1]]

0                                          achillotenotomy
1                                       acromioplasty open
2                                              acupuncture
3                                         adaptometry dark
4                                            adenoidectomy
5                                            adrenalectomy
6                                                 advocacy
7                                             alveolectomy
8                                              alveoplasty
9                                           amnio infusion
10                                          amnioreduction
11                                              amnioscopy
12                                       ampullectomy open
13                                              angioscopy
14                            anoplasty for anal stricture
15                                                anoscopy
16                                             aortograp

In [192]:
print(len(ACHI_processed))
ACHI_processed['ACHI'].nunique()

20759


20571

In [193]:
tfobj = TfidfVectorizer()
tf = tfobj.fit_transform(ACHI_processed['ACHI'])
ACHI_features = pd.DataFrame(tfobj.get_feature_names(), columns=['Features'])
ACHI_features.head()

,Features
0,aaa
1,abdomen
2,abdominal
3,abdomino
4,abdominoperineal


In [194]:
len(data_features[data_features['Features'].isin(ACHI_features['Features']) == True])

2165

In [195]:
# A function that uses dup_row_refrence by taking 2 DF and refrencing one by the other
def dup_ref_ACHI(DF_ref,DF_dup):
    #Dropping duplicate from refrence
    DF_unique =  DF_ref["ACHI"].drop_duplicates()
    #getting the subset of the services of processed DF that occur in refrence
    DF_dup = DF_dup[DF_dup["Service Name"].isin(DF_unique)]
    #applying dup_row_refrence for each row
    dup_ref = DF_dup['Service Name'].apply(lambda x: dup_row_refrence(x,DF_unique))
    return dup_ref

In [196]:
data_processed = create_document(data1,1)

In [197]:
dup_ref_ACHI(ACHI_processed, data_processed)

2          83
824      7877
1124      119
1470     7771
3453     7678
3456     7686
3899      825
3909      115
4245     2015
4650     7678
5099        5
7029     7948
8940     7633
9773       73
10015    8000
10194       4
11242      10
11636    7833
12129    7684
Name: Service Name, dtype: int64

In [198]:
ACHI_processed_ = dup_dup(ACHI_processed, 'ACHI')
print('len of duplicated services --> {}'.format(len(ACHI_processed_)))
print(len(collections.Counter(ACHI_processed['ACHI'])))
ACHI_processed_

len of duplicated services --> 188
20571


4019      4018
4022      4021
4100      4099
4101      4099
4102      4099
4282      4281
4330      4329
4351      4350
4799      4798
5694      5693
5695      5693
5697      5696
5698      5693
5702      5701
5703      5701
5705      5704
5706      5701
5844      5843
5848      5847
5857      5856
5879      5878
5932      5931
5933      5931
5934      5931
6009      6008
6522      6521
6849      6848
6850      6848
6852      6851
6853      6848
         ...  
19441    19431
19654    19653
19658    19657
19660    19659
19843    19842
19889    19888
19890    19888
19892    19891
19893    19888
20091    20090
20092    20090
20093    20090
20095    20094
20096    20094
20097    20094
20281    20280
20282    20280
20283    20280
20284    20280
20285    20280
20286    20280
20287    20280
20288    20280
20289    20280
20290    20280
20422    20421
20425    20424
20462    20461
20465    20464
20591    20590
Name: ACHI, Length: 188, dtype: int64

In [199]:
DF_data = pd.DataFrame()
DF_ref = pd.DataFrame()
DF_data['Name'] = data_processed['Service Name']
DF_ref['Name'] = ACHI_processed['ACHI']
DF = pd.concat(objs = [DF_data,DF_ref])
DF['Name'] = DF['Name'].apply(lambda x: ''.join(i for i in x if not i.isdigit()))
len(DF)

33242

In [200]:
stopWords = stopwords.words('english')
tfidf = TfidfVectorizer(stop_words=stopWords)

In [201]:
tfidf.fit(DF['Name'])
tfref = tfidf.transform(ACHI_processed['ACHI'])
tfdata = tfidf.transform(data_processed['Service Name'])
print(tfdata.shape)
print(tfref.shape)

(12483, 11795)
(20759, 11795)


In [202]:
%%time
result = pd.DataFrame(columns = ['Serv','cpt','Sim','Num'])
resultAll = pd.DataFrame(columns = ['Serv','cpt','Sim','Num'])
for i in range(len(data_processed)):
    for j in range(0,3):
        index, simlarity = find_similar(tfdata, tfref, i, j)
        result.loc[i] = [i, index, simlarity, j]
        resultAll = resultAll.append(result)
        result.drop(result.index, inplace = True)

Wall time: 5min 11s


In [203]:
len(resultAll[resultAll['Sim'] > 0.7])

897

In [204]:
result = resultAll.copy()
result['Serv'] = result['Serv'].astype(int)
result['cpt'] = result['cpt'].astype(int)
result['Num'] = result['Num'].astype(int)
result.head()

,Serv,cpt,Sim,Num
0,0,11146,0.241327,0
0,0,8515,0.209431,1
0,0,14998,0.201567,2
1,1,212,0.291546,0
1,1,5983,0.287152,1


In [205]:
first_pred = pd.DataFrame(columns=['Serv','cpt','Sim','Num'])
second_pred = pd.DataFrame(columns=['Serv','cpt','Sim','Num'])
third_pred = pd.DataFrame(columns=['Serv','cpt','Sim','Num'])

first_pred = result[result['Num'] == 0]
second_pred = result[result['Num'] == 1]
third_pred = result[result['Num'] == 2]

In [206]:
first_pred.columns = ['Serv', 'ACHI_1','Sim_1','Num']
second_pred.columns = ['Serv', 'ACHI_2','Sim_2','Num']
third_pred.columns = ['Serv', 'ACHI_3','Sim_3','Num']

In [207]:
AllResult = pd.concat(objs = [first_pred,second_pred, third_pred], axis=1, sort=False)
AllResult.head()

,Serv,ACHI_1,Sim_1,Num,Serv,ACHI_2,Sim_2,Num,Serv,ACHI_3,Sim_3,Num
0,0,11146,0.241327,0,0,8515,0.209431,1,0,14998,0.201567,2
1,1,212,0.291546,0,1,5983,0.287152,1,1,5982,0.279285,2
2,2,83,1.000000,0,2,775,0.831950,1,2,2219,0.812536,2
3,3,16507,0.832786,0,3,18799,0.734224,1,3,16506,0.700060,2
4,4,16507,0.380180,0,4,18799,0.335185,1,4,8907,0.310198,2


In [208]:
AllResult.drop(['Num'], axis=1, inplace=True)
AllResult.columns = ['Al Rajhi Service','ACHI_1','Sim_1','Serv','ACHI_2','Sim_2','Serv','ACHI_3','Sim_3']
AllResult.drop(['Serv'], axis=1, inplace=True)
AllResult.head()

,Al Rajhi Service,ACHI_1,Sim_1,ACHI_2,Sim_2,ACHI_3,Sim_3
0,0,11146,0.241327,8515,0.209431,14998,0.201567
1,1,212,0.291546,5983,0.287152,5982,0.279285
2,2,83,1.000000,775,0.831950,2219,0.812536
3,3,16507,0.832786,18799,0.734224,16506,0.700060
4,4,16507,0.380180,18799,0.335185,8907,0.310198


In [209]:
AllResult = AllResult.set_index('Al Rajhi Service')
AllResult = pd.concat(objs = [AllResult, data[['Service Code','Service Name','Service Type']]],axis = 1 , join_axes=[AllResult.index] )
AllResult.head()

,ACHI_1,Sim_1,ACHI_2,Sim_2,ACHI_3,Sim_3,Service Code,Service Name,Service Type
Al Rajhi Service,,,,,,,,,
0,11146,0.241327,8515,0.209431,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy
1,212,0.291546,5983,0.287152,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal
2,83,1.000000,775,0.831950,2219,0.812536,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal
3,16507,0.832786,18799,0.734224,16506,0.700060,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal
4,16507,0.380180,18799,0.335185,8907,0.310198,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal


In [210]:
AllResult = AllResult.reset_index()
AllResult = AllResult.set_index('ACHI_1')
AllResult = pd.concat(objs = [AllResult, ACHI[['ACHI','MBS','Block']]], axis = 1, join_axes=[AllResult.index])
AllResult = AllResult.reset_index()
AllResult.head()

,ACHI_1,Al Rajhi Service,Sim_1,ACHI_2,Sim_2,ACHI_3,Sim_3,Service Code,Service Name,Service Type,ACHI,MBS,Block
0,11146,0,0.241327,8515,0.209431,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602]
1,212,1,0.291546,5983,0.287152,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957]
2,83,2,1.000000,775,0.831950,2219,0.812536,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,Cholecystectomy,30443-00,[965]
3,16507,3,0.832786,18799,0.734224,16506,0.700060,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]
4,16507,4,0.380180,18799,0.335185,8907,0.310198,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]


In [211]:
AllResult.drop(['Al Rajhi Service'], axis=1, inplace=True)
AllResult.head()

,ACHI_1,Sim_1,ACHI_2,Sim_2,ACHI_3,Sim_3,Service Code,Service Name,Service Type,ACHI,MBS,Block
0,11146,0.241327,8515,0.209431,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602]
1,212,0.291546,5983,0.287152,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957]
2,83,1.000000,775,0.831950,2219,0.812536,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,Cholecystectomy,30443-00,[965]
3,16507,0.832786,18799,0.734224,16506,0.700060,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]
4,16507,0.380180,18799,0.335185,8907,0.310198,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]


In [212]:
AllResult.drop(['ACHI_1'],axis =1 ,inplace=True)
AllResult.head()

,Sim_1,ACHI_2,Sim_2,ACHI_3,Sim_3,Service Code,Service Name,Service Type,ACHI,MBS,Block
0,0.241327,8515,0.209431,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602]
1,0.291546,5983,0.287152,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957]
2,1.000000,775,0.831950,2219,0.812536,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,Cholecystectomy,30443-00,[965]
3,0.832786,18799,0.734224,16506,0.700060,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]
4,0.380180,18799,0.335185,8907,0.310198,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]


In [213]:
AllResult.columns = ['Score_1','ACHI_2','Score_2','ACHI_3','Score_3','Service Code','Service Name','Service Type','ACHI_1','MBS_1','Block_1']
AllResult.head()

,Score_1,ACHI_2,Score_2,ACHI_3,Score_3,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1
0,0.241327,8515,0.209431,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602]
1,0.291546,5983,0.287152,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957]
2,1.000000,775,0.831950,2219,0.812536,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,Cholecystectomy,30443-00,[965]
3,0.832786,18799,0.734224,16506,0.700060,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]
4,0.380180,18799,0.335185,8907,0.310198,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990]


In [214]:
AllResult['SCORE'] = AllResult['Score_1']
AllResult.drop(['Score_1'], axis=1, inplace=True)
AllResult.columns = ['ACHI_2','Score_2','ACHI_3','Score_3','Service Code','Service Name','Service Type','ACHI_1','MBS_1','Block_1','Score_1']
AllResult.head()

,ACHI_2,Score_2,ACHI_3,Score_3,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1,Score_1
0,8515,0.209431,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602],0.241327
1,5983,0.287152,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957],0.291546
2,775,0.831950,2219,0.812536,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,Cholecystectomy,30443-00,[965],1.000000
3,18799,0.734224,16506,0.700060,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990],0.832786
4,18799,0.335185,8907,0.310198,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,30614-02,[990],0.380180


In [215]:
AllResult = AllResult.set_index('ACHI_2')
AllResult = pd.concat(objs = [AllResult,ACHI[['ACHI','MBS','Block']] ], axis = 1, join_axes=[AllResult.index])
AllResult.head(2)

,Score_2,ACHI_3,Score_3,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1,Score_1,ACHI,MBS,Block
ACHI_2,,,,,,,,,,,,,
8515,0.209431,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602],0.241327,Injection (around) (into) (of) also Administ...,18360-00,[1552]
5983,0.287152,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957],0.291546,Biopsy ___duodenum ___closed (endoscopic) 3047...,30484-00,[957]


In [216]:
AllResult = AllResult.reset_index()
AllResult['SCORE'] = AllResult['Score_2']
AllResult.drop(['ACHI_2','Score_2'], axis=1, inplace=True)
AllResult.columns = ['ACHI_3','Score_3','Service Code','Service Name','Service Type','ACHI_1','MBS_1','Block_1','Score_1','ACHI_2','MBS_2','Block_2','Score_2']
AllResult.head(2)

,ACHI_3,Score_3,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1,Score_1,ACHI_2,MBS_2,Block_2,Score_2
0,14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602],0.241327,Injection (around) (into) (of) also Administ...,18360-00,[1552],0.209431
1,5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957],0.291546,Biopsy ___duodenum ___closed (endoscopic) 3047...,30484-00,[957],0.287152


In [217]:
AllResult = AllResult.set_index('ACHI_3')
AllResult = pd.concat(objs = [AllResult, ACHI[['ACHI','MBS','Block']] ], axis = 1, join_axes=[AllResult.index])
AllResult.head(2)

,Score_3,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1,Score_1,ACHI_2,MBS_2,Block_2,Score_2,ACHI,MBS,Block
ACHI_3,,,,,,,,,,,,,,,
14998,0.201567,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602],0.241327,Injection (around) (into) (of) also Administ...,18360-00,[1552],0.209431,Injection (around) (into) (of) also Administ...,30478-06,[851]
5982,0.279285,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957],0.291546,Biopsy ___duodenum ___closed (endoscopic) 3047...,30484-00,[957],0.287152,Biopsy ___duodenum ___closed (endoscopic) 3047...,30484-01,[957]


In [218]:
AllResult = AllResult.reset_index()
AllResult['SCORE'] = AllResult['Score_3']
AllResult.drop(['ACHI_3','Score_3'], axis=1, inplace=True)
AllResult.columns = ['Service Code','Service Name','Service Type','ACHI_1','MBS_1','Block_1','Score_1','ACHI_2','MBS_2','Block_2','Score_2','ACHI_3','MBS_3','Block_3','Score_3']
AllResult.head(2)

,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1,Score_1,ACHI_2,MBS_2,Block_2,Score_2,ACHI_3,MBS_3,Block_3,Score_3
0,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,Injection (around) (into) (of) also Administ...,30207-00,[1602],0.241327,Injection (around) (into) (of) also Administ...,18360-00,[1552],0.209431,Injection (around) (into) (of) also Administ...,30478-06,[851],0.201567
1,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,ERCP (endoscopic retrograde cholangiopancreato...,30484-00,[957],0.291546,Biopsy ___duodenum ___closed (endoscopic) 3047...,30484-00,[957],0.287152,Biopsy ___duodenum ___closed (endoscopic) 3047...,30484-01,[957],0.279285


In [225]:
AllResult = AllResult.sort_values(by = ['Score_1','Score_2','Score_3'], ascending=False)

In [226]:
len(AllResult[AllResult.Score_1 > 0.5])/len(AllResult)

0.13594488504365937

In [227]:
AllResult.head()

,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1,Score_1,ACHI_2,MBS_2,Block_2,Score_2,ACHI_3,MBS_3,Block_3,Score_3
45,07300271,INCISION OF BARTHOLIN GLAND (OR CYST) - 1 DAY PKG,Package Deal,Incision ___Bartholin's gland 35513-00 [1293] ...,35513-00,[1293],1.0,Incision ___Bartholin's gland,35513-00,[1293],0.923726,Drainage ___cyst ___Bartholin's gland,35513-00,[1293],0.831387
8605,07300548,UNILATERAL ORCHIDECTOMY - 1 DAY PKG,Package Deal,Orchidectomy (unilateral),30641-00,[1184],1.0,Orchidectomy (unilateral) 30641-00 [1184] ___b...,30641-01,[1184],0.894866,- - vulva 90485-00 [1344]___varicocele 30635-0...,30641-00,[1184],0.682828
7708,07300430,AMPUTATION OF TOE- 1 DAY PKG,Package Deal,Amputation ___toe,44338-00,[1533],1.0,Amputation ___foot ___toe,44338-00,[1533],0.839899,Amputation ___toe 44338-00 [1533] ___including...,44358-00,[1533],0.621651
10191,07300069,"REPAIR OF DIAPHRAGMATIC HERNIA, ABDOMINAL APPR...",Package Deal,Repair ___hernia ___diaphragmatic (hiatus) ___...,30601-00,[998],1.0,Repair ___hernia ___diaphragmatic (hiatus) ___...,30601-00,[998],0.798623,Repair ___hernia ___diaphragmatic (hiatus) ___...,30601-01,[998],0.751772
9418,05230099,MAXILLARY ARTERY TRANSANTRAL LIGATION,Surgical Procedure,Ligation ___artery 34106-14 [697] ___maxillary...,41707-00,[697],1.0,Clipping (of) ___artery 34106-14 [697] ___maxi...,41707-00,[697],0.777228,Interruption ___artery 34106-14 [697] ___maxil...,41707-00,[697],0.769328


In [228]:
AllResult_processed = create_document(AllResult,3)
AllResult_processed = create_document(AllResult_processed,7)
AllResult_processed = create_document(AllResult_processed,11)

In [229]:
AllResult_processed.head()

,Service Code,Service Name,Service Type,ACHI_1,MBS_1,Block_1,Score_1,ACHI_2,MBS_2,Block_2,Score_2,ACHI_3,MBS_3,Block_3,Score_3
45,07300271,INCISION OF BARTHOLIN GLAND (OR CYST) - 1 DAY PKG,Package Deal,incision bartholin s gland cyst,35513-00,[1293],1.0,incision bartholin s gland,35513-00,[1293],0.923726,drainage cyst bartholin s gland,35513-00,[1293],0.831387
8605,07300548,UNILATERAL ORCHIDECTOMY - 1 DAY PKG,Package Deal,orchidectomy unilateral,30641-00,[1184],1.0,orchidectomy unilateral bilateral,30641-01,[1184],0.894866,vulva varicocele with orchidectomy unila...,30641-00,[1184],0.682828
7708,07300430,AMPUTATION OF TOE- 1 DAY PKG,Package Deal,amputation toe,44338-00,[1533],1.0,amputation foot toe,44338-00,[1533],0.839899,amputation toe including metatarsal bone,44358-00,[1533],0.621651
10191,07300069,"REPAIR OF DIAPHRAGMATIC HERNIA, ABDOMINAL APPR...",Package Deal,repair hernia diaphragmatic hiatus abdominal a...,30601-00,[998],1.0,repair hernia diaphragmatic hiatus abdominal a...,30601-00,[998],0.798623,repair hernia diaphragmatic hiatus thoracic ap...,30601-01,[998],0.751772
9418,05230099,MAXILLARY ARTERY TRANSANTRAL LIGATION,Surgical Procedure,ligation artery maxillary transantral,41707-00,[697],1.0,clipping of artery maxillary transantral,41707-00,[697],0.777228,interruption artery maxillary transantral,41707-00,[697],0.769328


In [230]:
AllResult_processed.to_csv('result.csv')

In [123]:
# def get_index(x):
#     return service_name['Service Name'][service_name['Service Name'] == str(x)].index[0]

In [124]:
# def get_similar(l1,l2,description):
#     l3 = []
#     for i in l1:
#         l3.append(description.values[np.argmax([cosine_similarity(i,j) for j in l2])][0])
#     return l3    

In [125]:
# def find_similar(tfidf_data,tfidf_ref, index, rank = 1):
#     #multiplying the 2 matrix to find the cosine similarity
#     cosine_similarities = linear_kernel(tfidf_data[index:index+1], tfidf_ref[:]).flatten()
#     #getting the most similar index
#     highest_ind = cosine_similarities.argsort()[::-1][rank]
#     #return the index along with its similarity
#     return highest_ind, cosine_similarities[highest_ind]

In [126]:
# service_name.iloc[1][0]

In [127]:
# result = pd.DataFrame(columns = ['Serv','achi','Sim'])
# resultAll = pd.DataFrame(columns = ['Serv','achi','Sim'])
# for i in range(len(service_name)):
#     index, simlarity = find_similar(sn_vect, desc_vectorized, i, 1)
#     result.loc[i] = [service_name.iloc[i][0], achi.iloc[index][1], simlarity]
#     resultAll = resultAll.append(result)
#     result.drop(result.index, inplace = True)

In [128]:
# resultAll.columns = ['Service Name', 'Achi','Similarity']

In [129]:
# resultAll.head()

In [130]:
#get_similar(sn_vect,desc_vectorized,description)

In [131]:
#data['ACHI'].apply(lambda x : description.iloc[np.argmax([cosine_similarity(sn_vect[get_index(x)],j) for j in desc_vectorized]),0]).to_csv(r'D:\\Users\\Testing\\Desktop\\al-rajhai\\result.csv')

In [ ]:
# achi.iloc[1][1]